In [2]:
import os, sys, pathlib, time, subprocess
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import re

folder_path = pathlib.Path().resolve()
planing_path = folder_path.parent
problem_path = folder_path / "Problemas"
print("Current folder path:", folder_path)

step_pattern = re.compile(r"\s*\d+\s*:")

max_time = 60  # Tiempo máximo en segundos para cada resolución

if not problem_path.exists():
    os.makedirs(problem_path)

# Definir los escenarios a probar
scenarios = [
    {"desc": "1 persona, 1 caja", "params": {"-d": 1, "-k": 1, "-l": 2, "-p": 1, "-c": 1, "-g": 1, "-v": 0}},
    {"desc": "2 personas, 3 cajas", "params": {"-d": 1, "-k": 2, "-l": 3, "-p": 2, "-c": 3, "-g": 3, "-v": 0}},
]

results = []

for scenario in scenarios:
    desc = scenario["desc"]
    params = scenario["params"]
    # Construir el comando de generación del problema
    gen_cmd = f"python3 {folder_path}/generate_problem.py"
    for k, v in params.items():
        gen_cmd += f" {k} {v}"
    print(f"\n[INFO] Generando problema: {desc} ...")
    os.system(gen_cmd)
    # Buscar el archivo .pddl generado más reciente
    pddl_files = sorted(problem_path.glob("*.pddl"), key=os.path.getmtime, reverse=True)
    if not pddl_files:
        print("[ERROR] No se encontró el archivo .pddl generado.")
        continue
    problem_pddl = pddl_files[0]
    planificador_cmd = [
        f"{planing_path}/planificadores/ff",
        "-o", f"{folder_path}/dron-domain.pddl",
        "-f", str(problem_pddl)
    ]
    start = time.time()
    try:
        result = subprocess.run(planificador_cmd, timeout=max_time, capture_output=True, text=True)
        elapsed = time.time() - start
        if result.returncode == 0:
            result_lines = result.stdout.splitlines()
            step_count = 1
            for line in result.stdout.splitlines():
                if step_pattern.match(line):
                    step_count += 1
            print(f"[OK] Plan encontrado para '{desc}' en {elapsed:.4f} segundos con {step_count} pasos.")
            results.append({"escenario": desc, "tiempo": elapsed, "pasos": step_count})
        else:
            print(f"[ERROR] No se encontró plan para '{desc}'.")
    except subprocess.TimeoutExpired:
        print(f"[TIMEOUT] Tiempo excedido ({max_time}s) para '{desc}'.")
    os.remove(problem_pddl)

print("\n>>> Resultados de los escenarios probados:")
for r in results:
    print(f"Escenario: {r['escenario']}, Tiempo: {r['tiempo']:.4f}s, Pasos: {r['pasos']}")



Current folder path: /home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/Ejercicio1

[INFO] Generando problema: 1 persona, 1 caja ...


  File "/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/Ejercicio1/generate_problem.py", line 108
    default=2, help='capacidad máxima de cajas por dron (no usado en este dominio, se ignora)')
                                                                                              ^
SyntaxError: unmatched ')'


[TIMEOUT] Tiempo excedido (60s) para '1 persona, 1 caja'.


FileNotFoundError: [Errno 2] No such file or directory: '/home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/Ejercicio1/Problemas/drone_problem_d1_l118_p118_c118_g118_ct2.pddl'